In [8]:
import open3d as o3d
import numpy as np

import pandas
from sklearn.decomposition import PCA

import copy

In [9]:
# add aixs
aix_points = [[0, 0, 0],
              [0, 0, 100],
              [400, 0, 0],
              [0, 100, 0],
              [-400, 0, 0],]
aix_lines = [[0, 1],
             [0, 2],
             [0, 3]]

colors = [[1,0,1], [0,0,0], [0,0,0]]
aix_line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(aix_points),
    lines=o3d.utility.Vector2iVector(aix_lines),
)
aix_line_set.colors = o3d.utility.Vector3dVector(colors)

In [10]:
scan_obj = o3d.io.read_triangle_mesh("./data/femur_half_3.obj")
print(scan_obj)
points_center = scan_obj.get_center()
scan_obj.scale(1000.0, points_center)
number_of_points = np.asarray(scan_obj.vertices).shape[0]
scan_obj.compute_vertex_normals()
scan_pcd = scan_obj.sample_points_uniformly(number_of_points)
plane_model, inliers = scan_pcd.segment_plane(distance_threshold=5,
                                              ransac_n=3,
                                              num_iterations=1000)
[a, b, c, d] = plane_model
# floor
inlier_cloud = scan_pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

# bone 
bone_cloud = scan_pcd.select_by_index(inliers, invert=True)

cl, ind = bone_cloud.remove_statistical_outlier(nb_neighbors=10,std_ratio=2)

bone_cloud = bone_cloud.select_by_index(ind)

scan_bone_points_cloud_array = np.asarray(bone_cloud.points)

X = scan_bone_points_cloud_array
X

geometry::TriangleMesh with 35943 points and 70305 triangles.


array([[194.71503932,  29.26238589,  83.04554182],
       [195.84619502,  32.25221931,  80.3254979 ],
       [191.1990863 ,  29.59268001,  79.39898982],
       ...,
       [ 29.93830459,   1.38449136,  18.54640952],
       [ 35.25098112,   1.20028918,  17.36569241],
       [ 56.25212642,   1.55079809,  15.50840332]])

In [11]:
X = X - X.mean(axis=0, keepdims=True)
X

array([[188.13791195,   2.60889153,  46.74803253],
       [189.26906765,   5.59872494,  44.02798861],
       [184.62195894,   2.93918565,  43.10148053],
       ...,
       [ 23.36117723, -25.269003  , -17.75109977],
       [ 28.67385375, -25.45320518, -18.93181688],
       [ 49.67499905, -25.10269628, -20.78910597]])

In [12]:
pca = PCA(n_components=3)
pca.fit(X)

PCA(n_components=3)

In [13]:
pca.components_

array([[ 0.99959181, -0.02178979,  0.01847772],
       [-0.01773595,  0.03375758,  0.99927267],
       [ 0.02239771,  0.99919249, -0.03335734]])

In [14]:
x_ = pca.components_[0]
y_ =pca.components_[1]
z_ = np.cross(x_, y_)
z_

array([-0.02239771, -0.99919249,  0.03335734])

In [10]:
pca.explained_variance_

array([20519.00621512,   404.27366874,   170.46871303])

In [11]:
pca.explained_variance_ratio_

array([0.97275295, 0.01916557, 0.00808148])

In [52]:
X_after = pca.transform(X) # X_after = np.dot(pca.components_, X.T).T 
X_after

array([[188.88554145,  43.27077347,   5.30944142],
       [189.48884311,  41.047298  ,   7.75943821],
       [187.19210834,  39.71594111,   7.20603471],
       ...,
       [ 44.95433614, -21.81435346, -23.88564353],
       [ 46.57545543, -22.11680351, -23.79968968],
       [ 51.77939989, -22.80968669, -23.31513073]])

In [56]:
def distance(X, X_after, i, j):
    res1 =  (X[i][0] - X[j][0]) ** 2 + (X[i][1] - X[j][1]) ** 2 + (X[i][2] - X[j][2]) ** 2
    res2 =  (X_after[i][0] - X_after[j][0]) ** 2 + (X_after[i][1] - X_after[j][1]) ** 2 + (X_after[i][2] - X_after[j][2]) ** 2
    return res1 - res2

In [63]:
max_dis = 0
for i in range(1,4000,20):
    for j in range(1,4000,25):
        dis = distance(X, X_after, i, j)
        max_dis = max(dis, max_dis)
max_dis

1.4551915228366852e-11

In [14]:
X_pcd = o3d.geometry.PointCloud()
X_pcd.points = o3d.utility.Vector3dVector(X)

X_after_pcd = o3d.geometry.PointCloud()
X_after_pcd.points = o3d.utility.Vector3dVector(X_after)

In [15]:
X_pcd.paint_uniform_color([1, 0, 0])
X_after_pcd.paint_uniform_color([0, 1, 0])

geometry::PointCloud with 4654 points.

In [64]:
o3d.visualization.draw_geometries([X_pcd, aix_line_set])

In [71]:
o3d.visualization.draw_geometries([X_pcd,X_after_pcd, aix_line_set])

### -----------------

In [73]:
vectors = np.array([[0.99959866, -0.02154738,  0.01839112],
                    [0.01759935,  -0.03637623,  -0.99918318],
                    [0.02219878,  0.99910584, -0.03598241]])

In [85]:
res = np.dot(vectors, X.T).T
# res[:,2] = -res[:, 2]
res

array([[188.88554145,  43.27077347,  -5.30944142],
       [189.48884311,  41.047298  ,  -7.75943821],
       [187.19210834,  39.71594111,  -7.20603471],
       ...,
       [ 44.95433614, -21.81435346,  23.88564353],
       [ 46.57545543, -22.11680351,  23.79968968],
       [ 51.77939989, -22.80968669,  23.31513073]])

In [86]:
res_pcd = o3d.geometry.PointCloud()
res_pcd.points = o3d.utility.Vector3dVector(res)

In [ ]:
o3d.visualization.draw_geometries([res_pcd, aix_line_set])

In [83]:
o3d.visualization.draw_geometries([X_pcd,res_pcd, aix_line_set])

In [78]:
o3d.visualization.draw_geometries([X_after_pcd,res_pcd, aix_line_set])